# 10장. 데이터 다루기(Data Handling)

###   

## 10.1 데이터 검색하기

###   


### 10.1.1 1차원 데이터 탐색하기
: 우선 1차원 데이터가 주어졌을 땐 가장 먼저 요약 통계치(Summary Statistics)를 계산해봐야한다.

- 우선 히스토그램을 만들어보고자 한다.

In [ ]:
def bucketize(point, bucket_size):
    """각 데이터를 bucket_size의 배수에 해당하는 구간에 위치시킨다."""
    